In [2]:
import numpy as np
from families import Gaussian, Bernoulli, Poisson
from glm import GLM

import statsmodels.api as sm
import statsmodels

/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
N = 10000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = np.random.uniform(size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

## Linear Model

In [4]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

AttributeError: 'Gaussian' object has no attribute 'penalized_deviance'

In [ ]:
model.coef_

In [ ]:
model.parameter_covariance_

In [ ]:
np.sqrt(np.diag(model.parameter_covariance_))

In [ ]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

## Linear Model with Sample Weights

In [ ]:
sample_weights = np.random.uniform(0, 2, size=N)

In [ ]:
model = GLM(family=Gaussian())
model = model.fit(X, y, sample_weights=sample_weights)

In [ ]:
model.coef_

## Logistic Model

In [ ]:
p = 1 / (1 + np.exp(-nu))
y_logistic = np.random.binomial(1, p=p, size=N)

In [ ]:
model = GLM(family=Bernoulli())
model.fit(X, y_logistic)

In [ ]:
model.coef_

In [ ]:
model.dispersion_

In [ ]:
model.parameter_covariance_

In [ ]:
np.sqrt(np.diag(model.parameter_covariance_) / model.dispersion_)

In [ ]:
mod = sm.Logit(y_logistic, X)
res = mod.fit()
print(res.summary())

In [ ]:
coefs = np.empty((1000, 3))
for i in range(1000):
    y_logistic = np.random.binomial(1, p=p, size=N)
    model = GLM(family=Bernoulli())
    model.fit(X, y_logistic)
    coefs[i, :] = model.coef_
    
print(coefs.std(axis=0))

## Poission Model

In [ ]:
mu = np.exp(nu)
y_poisson = np.random.poisson(lam=mu, size=N)

In [ ]:
model = GLM(family=Poisson())
model.fit(X, y_poisson)

In [ ]:
model.coef_

In [ ]:
model.parameter_covariance_

In [ ]:
np.sqrt(np.diag(model.parameter_covariance_))

In [ ]:
mod = statsmodels.discrete.discrete_model.Poisson(y_poisson, X)
res = mod.fit()
print(res.summary())

## Poisson with Exposures

In [ ]:
mu = np.exp(nu)
expos = np.random.uniform(0, 10, size=N)
y_poisson = np.random.poisson(lam=(mu*expos), size=N)

In [ ]:
model = GLM(family=Poisson())
model.fit(X, y_poisson, offset=np.log(expos))

In [ ]:
model.coef_

## Linear Model with Correlated Predictors

In [ ]:
N = 1000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = 0.9*X[:, 1] + np.random.uniform(-0.1, 0.1, size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

In [ ]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [ ]:
model.coef_

In [ ]:
model.parameter_covariance_